In [1]:
import pandas as pd
from six import text_type


In [3]:
my_result = pd.read_csv("../../data/primer-finder-result.csv", sep=";")

my_result['f_index'] = my_result['f_index'].astype(int)
my_result['b_index'] = my_result['b_index'].astype(int)
my_result['f_match'] = my_result['f_match'].astype(str)

my_result['region_length'] = my_result['read'].str.len()



,BOLD ID,Read ID,Country,Phylum,Class,Order,Family,Genus,Species,f_score,f_match,f_index,b_score,b_match,b_index,read,possible_orfs,region_length
0,>BOLD:AAA7085,BLPDA1021-18,Costa_Rica,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Saturniidae,g:Lonomia,s:Lonomia_santarosensis,0,nan,370,46,CCTGTTTTAGCAGGAGCTATTAC,536,TCTAGCTATTTTTTCTTTACATTTAGCCGGTATTTCTTCAATTTTA...,"[0, 2]",166
1,>BOLD:AAA7085,MHMYN6354-14,Costa_Rica,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Saturniidae,g:Lonomia,s:Lonomia_santarosensis,0,nan,384,46,CCTGTTTTAGCAGGAGCTATTAC,550,TCTAGCTATTTTTTCTTTACATTTAGCCGGTATTTCTTCAATTTTA...,"[0, 2]",166
2,>BOLD:AAA7085,MHASB734-07,Costa_Rica,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Saturniidae,g:Lonomia,s:Lonomia_santarosensis,0,nan,384,46,CCTGTTTTAGCAGGAGCTATTAC,550,TCTAGCTATTTTTTCTTTACATTTAGCCGGTATTTCTTCAATTTTA...,"[0, 2]",166
3,>BOLD:AAA7085,MHATB310-06,Costa_Rica,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Saturniidae,g:Lonomia,s:Lonomia_santarosensis,0,nan,384,46,CCTGTTTTAGCAGGAGCTATTAC,550,TCTAGCTATTTTTTCTTTACATTTAGCCGGTATTTCTTCAATTTTA...,"[0, 2]",166
4,>BOLD:AAA7085,BLPAA6663-17,Costa_Rica,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Saturniidae,g:Lonomia,s:Lonomia_santarosensis,0,nan,384,46,CCTGTTTTAGCAGGAGCTATTAC,550,TCTAGCTATTTTTTCTTTACATTTAGCCGGTATTTCTTCAATTTTA...,"[0, 2]",166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374017,>BOLD:AAG6731,ANICL817-10,Australia,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Nolidae,g:Nola,s:Nola_biguttalis,52,GGAACAGGATGAACAGTTTACCCCCC,319,22,ATTACT-GCT-TTTCTTTTAC,520,GCTTTCATCTAATATTGCTCATAGAGGAAGATCTGTAGATTTAGCA...,"[0, 2]",175
2374018,>BOLD:ADC5281,ASGLY062-16,Ecuador,tax=p:Arthropoda,c:Insecta,o:Hymenoptera,f:Braconidae,g:Glyptapanteles,s:Glyptapanteles_jerrypowelli,52,GGAACAGGTTGAACAGTTTATCCTCC,231,46,CCAGTTTTAGCTGGTGCAATTAC,462,TTTATCATTAATTTTAGGACATGGTGGAATATCTGTTGATTTAGGA...,"[0, 2]",205
2374019,>BOLD:AEP3771,BHULP614-22,Bhutan,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Notodontidae,g:Gargetta,s:Gargetta_costigera,52,GGGACAGGATGAACTGTTTACCCTCC,319,43,CCAGTTTTAGCGGGAGCTATTAC,550,TTTATCATCAAACATCGCTCATGGTGGAAGATCTGTTGATTTAGCT...,"[0, 2]",205
2374020,>BOLD:ACQ1386,GBCL21575-14,NaN,tax=p:Arthropoda,c:Insecta,o:Coleoptera,f:Leiodidae,g:Phacomorphus,s:Phacomorphus_alexinae,52,GGAACTGGTTGAACTGTATACCCACC,319,46,CCTGTTTTAGCAGGAGCTATTAC,550,CTTATCTGCCAATATCTCCCATAGAGGATCTTCTGTTGATTTGGCT...,"[0, 2]",205


In [74]:
#params
threshold = 4

#setup
remaining_results = my_result[~my_result['possible_orfs'].str.contains(r'\[\]')]
solved_results = remaining_results[~remaining_results['possible_orfs'].str.contains(',')]
remaining_results = remaining_results[~(remaining_results['Read ID'].isin(solved_results['Read ID']))]

failed = 0

while remaining_results.size > 0:
    fam = remaining_results.Family.values[0]
    fam_comp_group = solved_results[solved_results['Family'] == fam]
    fg_size = len(fam_comp_group)
    if fg_size < threshold:
        print(f"{fg_size} entries of Family {fam} is too small")
        ord = remaining_results.Order.values[0]
        ord_comp_group = solved_results[solved_results['Order'] == ord]
        og_size = len(ord_comp_group)
        if og_size < threshold:
            print(f"{og_size} entries of Family {ord} is too small as well!!!!!!!!!!!!!")
            cls = remaining_results.Class.values[0]
            cls_comp_group = solved_results[solved_results['Class'] == cls]
            cg_size = len(cls_comp_group)
            if cg_size < threshold:
                print(f"{cg_size} entries of Class {cls} is too small. Nothing to do.")
                print(f"Removing Family '{fam}' anyways to continue.")
                remaining_results = remaining_results[~(remaining_results['Family'] == fam)]
                failed += fg_size
            else:
                remaining_results = remaining_results[~(remaining_results['Class'] == cls)]
        else:
            remaining_results = remaining_results[~(remaining_results['Order'] == ord)]

        print("-------------")
    else:
        remaining_results = remaining_results[~(remaining_results['Family'] == fam)]
print(f"A total of {failed} entries were impossible to match.")

3 entries of Family f:Saturniidae is too small
-------------
0 entries of Family f:Fanniidae is too small
-------------
1 entries of Family f:Ptiliidae is too small
-------------
0 entries of Family f:Flatidae is too small
-------------
3 entries of Family f:Psocidae is too small
-------------
0 entries of Family f:Pollicipedidae is too small
0 entries of Family o:Pollicipedomorpha is too small as well!!!!!!!!!!!!!
-------------
0 entries of Family f:Aetideidae is too small
-------------
0 entries of Family f:Krendowskiidae is too small
-------------
0 entries of Family f:Odontoceridae is too small
-------------
0 entries of Family f:Tridactylidae is too small
-------------
1 entries of Family f:Polybiidae is too small
-------------
3 entries of Family f:Caligidae is too small
-------------
0 entries of Family f:Aphelinidae is too small
-------------
0 entries of Family f:Boreidae is too small
-------------
1 entries of Family f:Dysderidae is too small
-------------
4 entries of Family